In [81]:
import pandas as pd
import numpy as np
import ast
import datetime
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
# from sklearn.decomposition import PCA

# 데이터 확인

In [82]:
port = pd.read_csv("C:/Users/minji/전민정ㅎ/실전 프로젝트/archive (8)/portfolio.csv",index_col=0)
port.head()

,reward,channels,difficulty,duration,offer_type,id
0,10,"['email', 'mobile', 'social']",10,7,bogo,ae264e3637204a6fb9bb56bc8210ddfd
1,10,"['web', 'email', 'mobile', 'social']",10,5,bogo,4d5c57ea9a6940dd891ad53e9dbe8da0
2,0,"['web', 'email', 'mobile']",0,4,informational,3f207df678b143eea3cee63160fa8bed
3,5,"['web', 'email', 'mobile']",5,7,bogo,9b98b8c7a33c4b65b9aebfe6a799e6d9
4,5,"['web', 'email']",20,10,discount,0b1e1539f2cc45b7b9fa7c272da2e1d7


In [83]:
profile = pd.read_csv("C:/Users/minji/전민정ㅎ/실전 프로젝트/archive (8)/profile.csv",index_col=0)
profile.head()

,gender,age,id,became_member_on,income
0,NaN,118,68be06ca386d4c31939f3a4f0e3dd783,20170212,NaN
1,F,55,0610b486422d4921ae7d2bf64640c50b,20170715,112000.0
2,NaN,118,38fe809add3b4fcf9315a9694bb96ff5,20180712,NaN
3,F,75,78afa995795e4d85b5d9ceeca43f5fef,20170509,100000.0
4,NaN,118,a03223e636434f42ac4c3df47e8bac43,20170804,NaN


In [84]:
trans = pd.read_csv("C:/Users/minji/전민정ㅎ/실전 프로젝트/archive (8)/transcript.csv",index_col=0)
trans.head()

,person,event,value,time
0,78afa995795e4d85b5d9ceeca43f5fef,offer received,{'offer id': '9b98b8c7a33c4b65b9aebfe6a799e6d9'},0
1,a03223e636434f42ac4c3df47e8bac43,offer received,{'offer id': '0b1e1539f2cc45b7b9fa7c272da2e1d7'},0
2,e2127556f4f64592b11af22de27a7932,offer received,{'offer id': '2906b810c7d4411798c6938adc9daaa5'},0
3,8ec6ce2a7e7949b1bf142def7d0e0586,offer received,{'offer id': 'fafdcd668e3743c1bb461111dcafc2a4'},0
4,68617ca6246f4fbc85e91a2a49552598,offer received,{'offer id': '4d5c57ea9a6940dd891ad53e9dbe8da0'},0


In [85]:
port.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10 entries, 0 to 9
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   reward      10 non-null     int64 
 1   channels    10 non-null     object
 2   difficulty  10 non-null     int64 
 3   duration    10 non-null     int64 
 4   offer_type  10 non-null     object
 5   id          10 non-null     object
dtypes: int64(3), object(3)
memory usage: 560.0+ bytes


In [86]:
profile.info()

<class 'pandas.core.frame.DataFrame'>
Index: 17000 entries, 0 to 16999
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   gender            14825 non-null  object 
 1   age               17000 non-null  int64  
 2   id                17000 non-null  object 
 3   became_member_on  17000 non-null  int64  
 4   income            14825 non-null  float64
dtypes: float64(1), int64(2), object(2)
memory usage: 796.9+ KB


In [87]:
trans.info()

<class 'pandas.core.frame.DataFrame'>
Index: 306534 entries, 0 to 306533
Data columns (total 4 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   person  306534 non-null  object
 1   event   306534 non-null  object
 2   value   306534 non-null  object
 3   time    306534 non-null  int64 
dtypes: int64(1), object(3)
memory usage: 11.7+ MB


In [88]:
profile.isnull().sum()

gender              2175
age                    0
id                     0
became_member_on       0
income              2175
dtype: int64

# 고객 분류

## 전처리

In [89]:
trans['value'] = trans['value'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)
trans['amount'] = trans['value'].apply(lambda x: x.get('amount') if isinstance(x, dict) else None)
trans['amount'] = trans['amount'].astype(float)
trans['offer_id'] = trans['value'].apply(lambda x: x.get('offer id') or x.get('offer_id') if isinstance(x, dict) else None)

In [90]:
# trans['keys'] = trans['value'].apply(lambda x: list(x.keys()) if isinstance(x, dict) else None)
# trans['keys_str'] = trans['keys'].apply(lambda x: str(sorted(x)) if isinstance(x, list) else str(x))
# trans['keys_str'].unique()

In [91]:
port['channels'] = port['channels'].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)
port['web'] = port['channels'].apply(lambda x: 1 if isinstance(x, list) and 'web' in x else 0)
port['email'] = port['channels'].apply(lambda x: 1 if isinstance(x, list) and 'email' in x else 0)
port['mobile'] = port['channels'].apply(lambda x: 1 if isinstance(x, list) and 'mobile' in x else 0)
port['social'] = port['channels'].apply(lambda x: 1 if isinstance(x, list) and 'social' in x else 0)

In [92]:
income_median = profile['income'].median()
profile['income'].fillna(income_median, inplace=True)
profile['income']

0         64000.0
1        112000.0
2         64000.0
3        100000.0
4         64000.0
           ...   
16995     54000.0
16996     72000.0
16997     73000.0
16998     50000.0
16999     82000.0
Name: income, Length: 17000, dtype: float64

In [93]:
profile['became_member_on'] = pd.to_datetime(profile['became_member_on'], format='%Y%m%d')
profile['became_member_on'] = profile['became_member_on'].dt.date

In [94]:
trans.drop(columns=['value'], axis=1, inplace=True)
port.drop(columns=['channels'], axis=1, inplace=True)

## 병합 전 확인

In [95]:
profile['id'] # 고객에 대한 인구통계학적 정보

0        68be06ca386d4c31939f3a4f0e3dd783
1        0610b486422d4921ae7d2bf64640c50b
2        38fe809add3b4fcf9315a9694bb96ff5
3        78afa995795e4d85b5d9ceeca43f5fef
4        a03223e636434f42ac4c3df47e8bac43
                       ...               
16995    6d5f3a774f3d4714ab0c092238f3a1d7
16996    2cb4f97358b841b9a9773a7aa05a9d77
16997    01d26f638c274aa0b965d24cefe3183f
16998    9dc1421481194dcd9400aec7c9ae6366
16999    e4052622e5ba45a8b96b59aba68cf068
Name: id, Length: 17000, dtype: object

In [96]:
trans['offer_id'] # 각 고객들이 받은 프로모션 데이터

0         9b98b8c7a33c4b65b9aebfe6a799e6d9
1         0b1e1539f2cc45b7b9fa7c272da2e1d7
2         2906b810c7d4411798c6938adc9daaa5
3         fafdcd668e3743c1bb461111dcafc2a4
4         4d5c57ea9a6940dd891ad53e9dbe8da0
                        ...               
306529                                None
306530                                None
306531                                None
306532                                None
306533                                None
Name: offer_id, Length: 306534, dtype: object

In [97]:
port['id'] # 프로모션 제공에 대한 정보

0    ae264e3637204a6fb9bb56bc8210ddfd
1    4d5c57ea9a6940dd891ad53e9dbe8da0
2    3f207df678b143eea3cee63160fa8bed
3    9b98b8c7a33c4b65b9aebfe6a799e6d9
4    0b1e1539f2cc45b7b9fa7c272da2e1d7
5    2298d6c36e964ae4a3e7e9706d1fb8c2
6    fafdcd668e3743c1bb461111dcafc2a4
7    5a8bc65990b245e5a138643cd4eb9837
8    f19421c1d4aa40978ebb69ca19b0e20d
9    2906b810c7d4411798c6938adc9daaa5
Name: id, dtype: object

In [98]:
# trans의 고객 id, profile의 id, portfoloi의 id 
# trans, port를 합칠 때 left로 하면 port에서 id빼고 나머지 컬럼 다 없어짐

trans_profile = pd.merge(trans, profile, how='left', left_on = 'person', right_on='id')
df = pd.merge(trans_profile, port, how='left', left_on = 'offer_id', right_on='id')
df.rename(columns = {'id_x': 'prof_id', 'id_y':'port_id'}, inplace = True)

In [99]:
trans_profile

,person,event,time,amount,offer_id,gender,age,id,became_member_on,income
0,78afa995795e4d85b5d9ceeca43f5fef,offer received,0,NaN,9b98b8c7a33c4b65b9aebfe6a799e6d9,F,75,78afa995795e4d85b5d9ceeca43f5fef,2017-05-09,100000.0
1,a03223e636434f42ac4c3df47e8bac43,offer received,0,NaN,0b1e1539f2cc45b7b9fa7c272da2e1d7,NaN,118,a03223e636434f42ac4c3df47e8bac43,2017-08-04,64000.0
2,e2127556f4f64592b11af22de27a7932,offer received,0,NaN,2906b810c7d4411798c6938adc9daaa5,M,68,e2127556f4f64592b11af22de27a7932,2018-04-26,70000.0
3,8ec6ce2a7e7949b1bf142def7d0e0586,offer received,0,NaN,fafdcd668e3743c1bb461111dcafc2a4,NaN,118,8ec6ce2a7e7949b1bf142def7d0e0586,2017-09-25,64000.0
4,68617ca6246f4fbc85e91a2a49552598,offer received,0,NaN,4d5c57ea9a6940dd891ad53e9dbe8da0,NaN,118,68617ca6246f4fbc85e91a2a49552598,2017-10-02,64000.0
...,...,...,...,...,...,...,...,...,...,...
306529,b3a1272bc9904337b331bf348c3e8c17,transaction,714,1.59,None,M,66,b3a1272bc9904337b331bf348c3e8c17,2018-01-01,47000.0
306530,68213b08d99a4ae1b0dcb72aebd9aa35,transaction,714,9.53,None,M,52,68213b08d99a4ae1b0dcb72aebd9aa35,2018-04-08,62000.0
306531,a00058cf10334a308c68e7631c529907,transaction,714,3.61,None,F,63,a00058cf10334a308c68e7631c529907,2013-09-22,52000.0
306532,76ddbd6576844afe811f1a3c0fbb5bec,transaction,714,3.53,None,M,57,76ddbd6576844afe811f1a3c0fbb5bec,2016-07-09,40000.0


In [100]:
trans_profile.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 306534 entries, 0 to 306533
Data columns (total 10 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   person            306534 non-null  object 
 1   event             306534 non-null  object 
 2   time              306534 non-null  int64  
 3   amount            138953 non-null  float64
 4   offer_id          167581 non-null  object 
 5   gender            272762 non-null  object 
 6   age               306534 non-null  int64  
 7   id                306534 non-null  object 
 8   became_member_on  306534 non-null  object 
 9   income            306534 non-null  float64
dtypes: float64(2), int64(2), object(6)
memory usage: 23.4+ MB


In [101]:
df

,person,event,time,amount,offer_id,gender,age,prof_id,became_member_on,income,reward,difficulty,duration,offer_type,port_id,web,email,mobile,social
0,78afa995795e4d85b5d9ceeca43f5fef,offer received,0,NaN,9b98b8c7a33c4b65b9aebfe6a799e6d9,F,75,78afa995795e4d85b5d9ceeca43f5fef,2017-05-09,100000.0,5.0,5.0,7.0,bogo,9b98b8c7a33c4b65b9aebfe6a799e6d9,1.0,1.0,1.0,0.0
1,a03223e636434f42ac4c3df47e8bac43,offer received,0,NaN,0b1e1539f2cc45b7b9fa7c272da2e1d7,NaN,118,a03223e636434f42ac4c3df47e8bac43,2017-08-04,64000.0,5.0,20.0,10.0,discount,0b1e1539f2cc45b7b9fa7c272da2e1d7,1.0,1.0,0.0,0.0
2,e2127556f4f64592b11af22de27a7932,offer received,0,NaN,2906b810c7d4411798c6938adc9daaa5,M,68,e2127556f4f64592b11af22de27a7932,2018-04-26,70000.0,2.0,10.0,7.0,discount,2906b810c7d4411798c6938adc9daaa5,1.0,1.0,1.0,0.0
3,8ec6ce2a7e7949b1bf142def7d0e0586,offer received,0,NaN,fafdcd668e3743c1bb461111dcafc2a4,NaN,118,8ec6ce2a7e7949b1bf142def7d0e0586,2017-09-25,64000.0,2.0,10.0,10.0,discount,fafdcd668e3743c1bb461111dcafc2a4,1.0,1.0,1.0,1.0
4,68617ca6246f4fbc85e91a2a49552598,offer received,0,NaN,4d5c57ea9a6940dd891ad53e9dbe8da0,NaN,118,68617ca6246f4fbc85e91a2a49552598,2017-10-02,64000.0,10.0,10.0,5.0,bogo,4d5c57ea9a6940dd891ad53e9dbe8da0,1.0,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
306529,b3a1272bc9904337b331bf348c3e8c17,transaction,714,1.59,None,M,66,b3a1272bc9904337b331bf348c3e8c17,2018-01-01,47000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
306530,68213b08d99a4ae1b0dcb72aebd9aa35,transaction,714,9.53,None,M,52,68213b08d99a4ae1b0dcb72aebd9aa35,2018-04-08,62000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
306531,a00058cf10334a308c68e7631c529907,transaction,714,3.61,None,F,63,a00058cf10334a308c68e7631c529907,2013-09-22,52000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
306532,76ddbd6576844afe811f1a3c0fbb5bec,transaction,714,3.53,None,M,57,76ddbd6576844afe811f1a3c0fbb5bec,2016-07-09,40000.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [102]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 306534 entries, 0 to 306533
Data columns (total 19 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   person            306534 non-null  object 
 1   event             306534 non-null  object 
 2   time              306534 non-null  int64  
 3   amount            138953 non-null  float64
 4   offer_id          167581 non-null  object 
 5   gender            272762 non-null  object 
 6   age               306534 non-null  int64  
 7   prof_id           306534 non-null  object 
 8   became_member_on  306534 non-null  object 
 9   income            306534 non-null  float64
 10  reward            167581 non-null  float64
 11  difficulty        167581 non-null  float64
 12  duration          167581 non-null  float64
 13  offer_type        167581 non-null  object 
 14  port_id           167581 non-null  object 
 15  web               167581 non-null  float64
 16  email             16

In [103]:
# df[df['prof_id'].str.startswith('ff')]

In [104]:
port.to_csv('portfolio.csv',index=False)
trans.to_csv('transcript.csv',index=False)
profile.to_csv('profile.csv',index=False)

In [105]:
import sys
!{sys.executable} -m pip install mysql-connector-python


Defaulting to user installation because normal site-packages is not writeable


In [106]:
import mysql.connector

conn = mysql.connector.connect(
    host = "localhost",
    user="root",
    password="system",
    database="sparta"
)
# 커서 생성
cursor = conn.cursor()
# 쿼리 실행 예시
cursor.execute("SELECT portfolio.* from sparta.portfolio")
# 결과 출력
result = cursor.fetchone()
result


(10, 10, 7, 'bogo', 'ae264e3637204a6fb9bb56bc8210ddfd', 0, 1, 1, 1)

In [107]:
import mysql.connector

conn = mysql.connector.connect(
    host = "localhost",
    user="root",
    password="system",
    database="sparta"
)
# 커서 생성
cursor = conn.cursor()
# 쿼리 실행 예시
cursor.execute("SELECT transcript.*, profile.*, portfolio.*, profile.id AS prof_id, portfolio.id AS port_id FROM transcript LEFT JOIN profile ON transcript.person = profile.id LEFT JOIN portfolio ON transcript.offer_id = portfolio.id")
# 결과와 컬럼명 가져오기
rows = cursor.fetchall()
columns = [col[0] for col in cursor.description]
# DataFrame으로 변환
df = pd.DataFrame(rows, columns=columns)
df

,person,event,time,amount,offer_id,gender,age,id,became_member_on,income,...,difficulty,duration,offer_type,id,web,email,mobile,social,prof_id,port_id
0,78afa995795e4d85b5d9ceeca43f5fef,offer received,0,,9b98b8c7a33c4b65b9aebfe6a799e6d9,F,75,78afa995795e4d85b5d9ceeca43f5fef,2017-05-09,100000.0,...,5.0,7.0,bogo,9b98b8c7a33c4b65b9aebfe6a799e6d9,1.0,1.0,1.0,0.0,78afa995795e4d85b5d9ceeca43f5fef,9b98b8c7a33c4b65b9aebfe6a799e6d9
1,a03223e636434f42ac4c3df47e8bac43,offer received,0,,0b1e1539f2cc45b7b9fa7c272da2e1d7,,118,a03223e636434f42ac4c3df47e8bac43,2017-08-04,64000.0,...,20.0,10.0,discount,0b1e1539f2cc45b7b9fa7c272da2e1d7,1.0,1.0,0.0,0.0,a03223e636434f42ac4c3df47e8bac43,0b1e1539f2cc45b7b9fa7c272da2e1d7
2,e2127556f4f64592b11af22de27a7932,offer received,0,,2906b810c7d4411798c6938adc9daaa5,M,68,e2127556f4f64592b11af22de27a7932,2018-04-26,70000.0,...,10.0,7.0,discount,2906b810c7d4411798c6938adc9daaa5,1.0,1.0,1.0,0.0,e2127556f4f64592b11af22de27a7932,2906b810c7d4411798c6938adc9daaa5
3,8ec6ce2a7e7949b1bf142def7d0e0586,offer received,0,,fafdcd668e3743c1bb461111dcafc2a4,,118,8ec6ce2a7e7949b1bf142def7d0e0586,2017-09-25,64000.0,...,10.0,10.0,discount,fafdcd668e3743c1bb461111dcafc2a4,1.0,1.0,1.0,1.0,8ec6ce2a7e7949b1bf142def7d0e0586,fafdcd668e3743c1bb461111dcafc2a4
4,68617ca6246f4fbc85e91a2a49552598,offer received,0,,4d5c57ea9a6940dd891ad53e9dbe8da0,,118,68617ca6246f4fbc85e91a2a49552598,2017-10-02,64000.0,...,10.0,5.0,bogo,4d5c57ea9a6940dd891ad53e9dbe8da0,1.0,1.0,1.0,1.0,68617ca6246f4fbc85e91a2a49552598,4d5c57ea9a6940dd891ad53e9dbe8da0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
306529,3a34a62146b040e3ac783d58c2b69bfe,transaction,714,3.91,,M,33,3a34a62146b040e3ac783d58c2b69bfe,2017-08-28,35000.0,...,NaN,NaN,None,None,NaN,NaN,NaN,NaN,3a34a62146b040e3ac783d58c2b69bfe,None
306530,d93aac1763804476bac97bbf6b76017f,transaction,714,3.85,,M,23,d93aac1763804476bac97bbf6b76017f,2017-07-11,72000.0,...,NaN,NaN,None,None,NaN,NaN,NaN,NaN,d93aac1763804476bac97bbf6b76017f,None
306531,a4c2dd694269444085976f10b9b225f6,transaction,714,28.58,,M,66,a4c2dd694269444085976f10b9b225f6,2015-10-30,72000.0,...,NaN,NaN,None,None,NaN,NaN,NaN,NaN,a4c2dd694269444085976f10b9b225f6,None
306532,4b59509988d34c7298119a840c946453,transaction,714,0.06,,M,42,4b59509988d34c7298119a840c946453,2017-12-21,59000.0,...,NaN,NaN,None,None,NaN,NaN,NaN,NaN,4b59509988d34c7298119a840c946453,None
